# Modified Net Ratings

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/teams/"
box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"

team_dict, team_list = get_teams()

black1 = '#252525'
black2 = '#222222'

## Time Decay Net Rating

In [ ]:
df0 = get_box("T","Base",False,[2024])
df0 = df0[["game_id","game_date"]]
df0.columns = ["gameid","gamedate"]
df0["gameid"] = df0["gameid"].astype(int)
df1 = get_box("T","Adv",False,[2024])
df1 = pd.merge(df0,df1,on="gameid")
df1 = df1[['gameid', 'gamedate', 'teamid','teamtricode',
       'offensiverating', 'defensiverating', 'netrating', 
       'pace', 'possessions']]
df1["pts11"] = df1["offensiverating"] * df1["possessions"]
df1["pts12"] = df1["defensiverating"] * df1["possessions"]

In [ ]:
df2 = df1.groupby(["teamid","teamtricode"])[["possessions","pts11","pts12"]].sum()
df2 = df2.reset_index()
df2["offensiverating"] = round(df2["pts11"]/df2["possessions"],1)
df2["defensiverating"] = round(df2["pts12"]/df2["possessions"],1)
df2["netrating"] = round(df2["offensiverating"] - df2["defensiverating"],1)
df2 = df2.drop(columns=["possessions","pts11","pts12"])

In [ ]:
df3 = df1.sort_values(by=["teamtricode","gamedate"]).reset_index(drop=True)
df3["dated"] = (df3["gamedate"] - df3["gamedate"].iloc[-1]).dt.days
teams = df3["teamtricode"].unique()
scale = 1e-2
df3["tdecay"] = round(np.exp(scale*df3["dated"]),4)
df3["pts21"] = df3["pts11"]*df3["tdecay"]
df3["pts22"] = df3["pts12"]*df3["tdecay"]
df3["possessions2"] = df3["possessions"]*df3["tdecay"]

In [ ]:
df3[["gamedate","teamtricode","dated","tdecay"]]

In [ ]:
df4 = df3.groupby(["teamid","teamtricode"])[["possessions2","pts21","pts22"]].sum()
df4 = df4.reset_index()
df4["offensiverating"] = round(df4["pts21"]/df4["possessions2"],1)
df4["defensiverating"] = round(df4["pts22"]/df4["possessions2"],1)
df4["netrating"] = round(df4["offensiverating"] - df4["defensiverating"],1)
df4 = df4.drop(columns=["possessions2","pts21","pts22"])

In [ ]:
df5 = pd.merge(df2,df4,on=["teamid","teamtricode"],suffixes=["","_t"])
df5["netrating_d"] = df5["netrating_t"] - df5["netrating"]
df5["team"] = df5["teamid"].map(team_dict)
df5["image"] = img_DIR_T + df5["team"] + ".png"

In [ ]:
p = (
    ggplot(df5)
    + aes(x="netrating",y="netrating_d",image="image")
    + geom_image()
    + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
    + geom_vline(xintercept=0, linetype="dashed", color="red", size=0.5)
    + theme_idv
    + theme(
        figure_size=(8,8),
        plot_title=element_text(weight="bold",size=24),
        plot_subtitle=element_text(size=14),
    )
    + pnba
    + labs(
        x="Net Rating",
        y="Hotness",
        title="Team Hotness vs Net Rating",
        subtitle="2024-25 NBA Season, as of "+ datetime.today().strftime("%Y-%m-%d")+"\n"+"Hotness is the difference between Time-decay Net Rating and overall Net Rating"
    )
    
)
p

## Saturated Net Rating

In [ ]:
df1 = get_box("T","Adv",False,[2024])
df1 = df1[['gameid', 'teamid','teamtricode',
       'offensiverating', 'defensiverating', 'netrating', 
       'pace', 'possessions']]

In [ ]:
df1["netrating_s"] = df1["netrating"]
df1.loc[df1["netrating"]>20,"netrating_s"] = 20
df1.loc[df1["netrating"]<-20,"netrating_s"] = -20

In [ ]:
df1["net1p"] = df1["netrating"]*df1["possessions"]
df1["net2p"] = df1["netrating_s"]*df1["possessions"]

In [ ]:
df2 = df1.groupby(["teamid","teamtricode"])[["possessions","net1p","net2p"]].sum()
df2 = df2.reset_index()
df2["netrating"] = round(df2["net1p"]/df2["possessions"],1)
df2["netrating_s"] = round(df2["net2p"]/df2["possessions"],1)
df2["netrating_d"] = df2["netrating_s"] - df2["netrating"]
df2["team"] = df2["teamid"].map(team_dict)
df2["image"] = img_DIR_T + df2["team"] + ".png"

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
subtitle = ("2024-25 NBA Season, as of " +  r"$\bf{" + today + "}$" + "\n"+
r"$\bf{" +"Blowout\,Factor"+"}$ is the difference between Saturated Net Rating and overall Net Rating"+ "\n"+
r"$\bf{" + "Saturated\,Net\,Rating:" + "}$ Limiting maximum and minimum net rating at 20 and -20")

p = (
    ggplot(df2)
    + aes(x="netrating_s",y="netrating_d",image="image")
    + geom_image()
    + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
    + geom_vline(xintercept=0, linetype="dashed", color="red", size=0.5)
    + theme_idv
    + theme(
        figure_size=(8,8),
        plot_title=element_text(weight="bold",size=22),
        plot_subtitle=element_text(size=12),
    )
    + pnba
    + labs(
        x="Saturated Net Rating",
        y="Blowout Factor",
        title="Team Blowout Factor vs Saturated Net Rating",
        subtitle=subtitle
    )
)
p

## Saturated Net Rating v2

In [ ]:
df1 = get_box("T","Adv",False,[2024])
df1 = df1[['gameid', 'teamid','teamtricode',
       'offensiverating', 'defensiverating', 'netrating', 
       'pace', 'possessions']]

In [ ]:
pow = 0.8
df1["netrating_s"] = np.sign(df1["netrating"])*abs(df1["netrating"])**pow
# df1.loc[df1["netrating"]>20,"netrating_s"] = 20
# df1.loc[df1["netrating"]<-20,"netrating_s"] = -20

In [ ]:
df1[["netrating","netrating_s"]].query("netrating>20")

In [ ]:
df1["net1p"] = df1["netrating"]*df1["possessions"]
df1["net2p"] = df1["netrating_s"]*df1["possessions"]

In [ ]:
df2 = df1.groupby(["teamid","teamtricode"])[["possessions","net1p","net2p"]].sum()
df2 = df2.reset_index()
df2["netrating"] = round(df2["net1p"]/df2["possessions"],1)
df2["netrating_s"] = round(df2["net2p"]/df2["possessions"],1)
df2["netrating_d"] = df2["netrating_s"] - df2["netrating"]
df2["team"] = df2["teamid"].map(team_dict)
df2["image"] = img_DIR_T + df2["team"] + ".png"

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
subtitle = ("2024-25 NBA Season, as of " +  r"$\bf{" + today + "}$" + "\n"+
r"$\bf{" +"Blowout\,Factor"+"}$ is the difference between Saturated Net Rating and overall Net Rating"+ "\n"+
r"$\bf{" + "Saturated\,Net\,Rating:" + "}$ Limiting maximum and minimum net rating at 20 and -20")

p = (
    ggplot(df2)
    + aes(x="netrating_s",y="netrating_d",image="image")
    + geom_image()
    + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
    + geom_vline(xintercept=0, linetype="dashed", color="red", size=0.5)
    + theme_idv
    + theme(
        figure_size=(8,8),
        plot_title=element_text(weight="bold",size=22),
        plot_subtitle=element_text(size=12),
    )
    + pnba
    + labs(
        x="Saturated Net Rating",
        y="Blowout Factor",
        title="Team Blowout Factor vs Saturated Net Rating",
        subtitle=subtitle
    )
)
p

# Ratings Movement

## Ratings Removing the Best and Worst Games

In [ ]:
year = 2024
df1 = pd.read_parquet(box_DIR + f"NBA_Box_T_Adv_{year}.parquet")
df1 = df1.rename(columns={"teamTricode":"Team"})

In [ ]:
n = 5
df2 = df1.groupby("Team")
keys = list(df2.groups)
rat1,orat1,drat1 = [],[],[]
rat2,orat2,drat2 = [],[],[]
for key in keys:
    df3 = df2.get_group(key).sort_values("netRating",ascending=False).iloc[n:-n].reset_index(drop=True)
    df4 = df2.get_group(key).sort_values("netRating",ascending=False).reset_index(drop=True)
    rat1.append(round((df3["netRating"]*df3["possessions"]).sum()/df3["possessions"].sum(),2))
    rat2.append(round((df4["netRating"]*df4["possessions"]).sum()/df4["possessions"].sum(),2))
    orat1.append(round((df3["offensiveRating"]*df3["possessions"]).sum()/df3["possessions"].sum(),2))
    orat2.append(round((df4["offensiveRating"]*df4["possessions"]).sum()/df4["possessions"].sum(),2))
    drat1.append(round((df3["defensiveRating"]*df3["possessions"]).sum()/df3["possessions"].sum(),2))
    drat2.append(round((df4["defensiveRating"]*df4["possessions"]).sum()/df4["possessions"].sum(),2))

In [ ]:
df5 = pd.DataFrame({"teamTricode":keys,"Net_Rating":rat1,"mNet_rating":rat2,"Off_Rating":orat1,"mOff_Rating":orat2,"Def_Rating":drat1,"mDef_Rating":drat2})
df5["Off_R"] = df5["Off_Rating"].rank(ascending=False  ,method="first").astype(int)
df5["Def_R"] = df5["Def_Rating"].rank(ascending=True   ,method="first").astype(int)
df5["Net_R"] = df5["Net_Rating"].rank(ascending=False  ,method="first").astype(int)
df5["mOff_R"] = df5["mOff_Rating"].rank(ascending=False  ,method="first").astype(int)
df5["mDef_R"] = df5["mDef_Rating"].rank(ascending=True   ,method="first").astype(int)
df5["mNet_R"] = df5["mNet_rating"].rank(ascending=False,method="first").astype(int)

In [ ]:
df6 = add_tinfo(df5,on="teamTricode")

In [ ]:
p = (ggplot(df6)
 # Slight modifications for the original lines,
 # 1. Nudge the text to either sides of the points
 # 2. Alter the color and alpha values
 + geom_text(aes(1, 'Net_R', label='nameTeam' ), nudge_x=-0.05, ha='right', size=10, color=black1)
 + geom_text(aes(2, 'mNet_R', label='nameTeam'), nudge_x=0.05, ha='left', size=10, color=black1)
 + geom_point(aes(1, 'Net_R', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_point(aes(2, 'mNet_R', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_segment(aes(x=1, y='Net_R', xend=2, yend='mNet_R', color='colorsTeam'), alpha=.7, size=2)
 + geom_image(aes(x=0.4, y='Net_R',image="image"),size=0.05)
 + geom_image(aes(x=2.6, y='mNet_R',image="image"),size=0.05)

 # Text Annotations
 #+ annotate('text', x=1, y=0, label='Rank in 1960', fontweight='bold', ha='right', size=10, color=black2)
 #+ annotate('text', x=2, y=0, label='Rank in 2015', fontweight='bold', ha='left', size=10, color=black2)
#  + annotate('text', x=1.5, y=0, label='Lines show change in rank', size=11, color=black1)
 + annotate('text', x=0.8, y=0, label='Net Rating Rank', size=11, color=black1)
 + annotate('text', x=2.2, y=0, label='Modified Net Rating Rank', size=11, color=black1)
 #+ annotate('label', x=1.5, y=3, label='Lower infant\ndeath rates', size=9, color=black1,
 #           label_size=0, fontstyle='italic')
 #+ annotate('label', x=1.5, y=33, label='Higher infant\ndeath rates', size=9, color=black1,
 #           label_size=0, fontstyle='italic')

 # Prevent country names from being chopped Net
 + lims(x=(0.35, 2.65))
#  + labs(color='Income Group')
 # Countries with lower rates on top
 + scale_y_reverse(breaks=np.arange(1,31,1))
 # Change colors
 + scale_color_identity(aesthetics=["color"],guide=None)
 # Removes all decorations
 + theme_idv
 # Changing the figure size prevents the country names from squishing up
 + theme(figure_size=(8, 11))
 + theme(
     axis_title_x = element_blank(),
     axis_text_x = element_blank(),
     plot_title=element_text(face="bold", size=20),
     panel_border=element_blank(),
     axis_ticks=element_blank(),
 )
 + labs(
        title=f"Net Rating vs Modified Net Rating",
        subtitle = f"{n} Best and {n} Worst games removed for Modified Net Rating",
        caption="@sradjoker | source:nba.com/stats",
        y=f"Net Rating",
 )
)
# p.save("../figs/team_leaders/m_Net_mvmt.png", dpi=300, verbose = False)
p

## Net Rating, Cap Max at +25, Min at -25

In [ ]:
year = 2024
df1 = pd.read_parquet(box_DIR + f"NBA_Box_T_Adv_{year}.parquet")
# df1 = df1.rename(columns={"teamTricode":"Team"})

In [ ]:
df2 = df1.groupby("teamTricode")
keys = list(df2.groups)
rat1,orat1,drat1 = [],[],[]
rat2,orat2,drat2 = [],[],[]
for key in keys:
    df3 = df2.get_group(key)
    df3.loc[df3["netRating"] > 25,"netRating"] = 25
    df3.loc[df3["netRating"] < -25,"netRating"] = -25
    df4 = df2.get_group(key).sort_values("netRating",ascending=False).reset_index(drop=True)
    rat1.append(round((df3["netRating"]*df3["possessions"]).sum()/df3["possessions"].sum(),2))
    rat2.append(round((df4["netRating"]*df4["possessions"]).sum()/df4["possessions"].sum(),2))

In [ ]:
df5 = pd.DataFrame({"teamTricode":keys,"Net_Rating":rat1,"mNet_rating":rat2,})
df5["Net_R"] = df5["Net_Rating"].rank(ascending=False  ,method="first").astype(int)
df5["mNet_R"] = df5["mNet_rating"].rank(ascending=False,method="first").astype(int)

In [ ]:
df6 = add_tinfo(df5,on="teamTricode")

In [ ]:
p = (ggplot(df6)
 # Slight modifications for the original lines,
 # 1. Nudge the text to either sides of the points
 # 2. Alter the color and alpha values
 + geom_text(aes(1, 'Net_R', label='nameTeam' ), nudge_x=-0.05, ha='right', size=10, color=black1)
 + geom_text(aes(2, 'mNet_R', label='nameTeam'), nudge_x=0.05, ha='left', size=10, color=black1)
 + geom_point(aes(1, 'Net_R', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_point(aes(2, 'mNet_R', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_segment(aes(x=1, y='Net_R', xend=2, yend='mNet_R', color='colorsTeam'), alpha=.7, size=2)
 + geom_image(aes(x=0.4, y='Net_R',image="image"),size=0.05)
 + geom_image(aes(x=2.6, y='mNet_R',image="image"),size=0.05)

 # Text Annotations
 #+ annotate('text', x=1, y=0, label='Rank in 1960', fontweight='bold', ha='right', size=10, color=black2)
 #+ annotate('text', x=2, y=0, label='Rank in 2015', fontweight='bold', ha='left', size=10, color=black2)
#  + annotate('text', x=1.5, y=0, label='Lines show change in rank', size=11, color=black1)
 + annotate('text', x=0.8, y=0, label='Net Rating Rank', size=11, color=black1)
 + annotate('text', x=2.2, y=0, label='Modified Net Rating Rank', size=11, color=black1)
 #+ annotate('label', x=1.5, y=3, label='Lower infant\ndeath rates', size=9, color=black1,
 #           label_size=0, fontstyle='italic')
 #+ annotate('label', x=1.5, y=33, label='Higher infant\ndeath rates', size=9, color=black1,
 #           label_size=0, fontstyle='italic')

 # Prevent country names from being chopped Net
 + lims(x=(0.35, 2.65))
#  + labs(color='Income Group')
 # Countries with lower rates on top
 + scale_y_reverse(breaks=np.arange(1,31,1))
 # Change colors
 + scale_color_identity(aesthetics=["color"],guide=None)
 # Removes all decorations
 + theme_idv
 # Changing the figure size prevents the country names from squishing up
 + theme(figure_size=(8, 11))
 + theme(
     axis_title_x = element_blank(),
     axis_text_x = element_blank(),
     plot_title=element_text(face="bold", size=24),
     plot_subtitle=element_text(size=18),
     panel_border=element_blank(),
     axis_ticks=element_blank(),
 )
 + labs(
        title=f"Net Rating vs Modified Net Rating",
        subtitle = f"Max and Min Net Rating capped at +25 and -25",
        caption="@sradjoker | source:nba.com/stats",
        y=f"Net Rating",
 )
)
# p.save("../figs/team_leaders/m_Net_mvmt.png", dpi=300, verbose = False)
p